# CT and PET Image Preparation

Images are provided as MATLAB files containing the tumor region in addition to a bounding region containing 20 voxels of the original image in the x, and y direction, and approximately 20 voxels in the z direction. 

In [1]:
import sys
sys.path.append('./../src/')

import ioutil

## Prep original images

Prepare the cropped images displaying the tumor and a set of border voxels by converting MATLAB files to NRRD. Note that the masks were binary.

In [3]:
ioutil.matlab_to_nrrd(
    './../../data/images/ct_cropped_raw/',
    './../../data/images/ct_cropped/', modality='CT'
)
ioutil.matlab_to_nrrd(
    './../../data/images/pet_cropped_raw/',
    './../../data/images/pet_cropped/', modality='PET'
)
ioutil.matlab_to_nrrd(
    './../../data/images/masks_cropped_raw/',
    './../../data/images/masks_cropped/', modality='mask'
)

In [4]:
num_ct = len(
    ioutil.relative_paths(
        './../../data/images/ct_cropped/', target_format='nrrd'
    )
)
num_pet = len(
    ioutil.relative_paths(
        './../../data/images/masks_cropped/', target_format='nrrd'
    )
)
num_masks = len(
    ioutil.relative_paths(
        './../../data/images/pet_cropped/', target_format='nrrd'
    )
)
num_ct, num_pet, num_masks

(198, 198, 198)

## Prep histogram equalized images

In [5]:
import numpy as np

In [6]:
def histogram_equal(image, nbins=512, **kwargs):
    
    _image = np.copy(image)
    
    # get image histogram
    image_histogram, bins = np.histogram(_image.flatten(), nbins)
    
    # Cumulative distribution function.
    cdf = image_histogram.cumsum() 
    # Normalize.
    cdf = np.max(_image) * cdf / cdf[-1] 
    # Use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)

    return image_equalized.reshape(np.shape(_image))

In [7]:
ioutil.matlab_to_nrrd(
    './../../data/images/ct_cropped_raw/',
    './../../data/images/ct_cropped_histeq/', 
    transform=histogram_equal, modality='CT', nbins=512
)
ioutil.matlab_to_nrrd(
    './../../data/images/pet_cropped_raw/',
    './../../data/images/pet_cropped_histeq/',
    transform=histogram_equal, modality='PET', nbins=256
)

In [8]:
num_ct = len(
    ioutil.relative_paths(
        './../../data/images/ct_cropped_histeq/', target_format='nrrd'
    )
)
num_pet = len(
    ioutil.relative_paths(
        './../../data/images/pet_cropped_histeq/', target_format='nrrd'
    )
)
num_ct, num_pet

RuntimeError: Invalid path ./../../data/images/masks_cropped_histeq/